In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [ ]:
data = pd.read_csv('images.csv')

In [ ]:
data = data[data['kids'] == False]
data = data[['image', 'label']]

In [ ]:
image_dir = 'images_original/'

In [ ]:
img_size = (100, 100)

In [ ]:
from PIL import Image
import os
import numpy as np

def load_images(file_paths, img_size):
    images = []
    failed_images = []
    for file_path in file_paths:
        file_path_without_extension = os.path.splitext(file_path)[0]  # Remove the file extension
        full_path = os.path.join(image_dir, f"{file_path_without_extension}.jpg")
        try:
            with Image.open(full_path) as img:
                img = img.convert('RGB')
                img = img.resize(img_size)
                img_array = np.array(img)
                images.append(img_array)
        except Exception as e:
            print(f"Error loading image: {file_path}. Exception: {e}")
            failed_images.append(file_path)
    return np.array(images), failed_images

# Update this variable with the correct directory
image_dir = 'images_compressed'  # Adjust if needed
image_paths = data['image'].values  # Assuming this contains all image file names

# Assuming img_size is defined somewhere (e.g., img_size = (224, 224))

loaded_images, failed_list = load_images(image_paths, img_size)

print(f"Successfully loaded {len(loaded_images)} images.")
print(f"Failed to load {len(failed_list)} images.")
